In [10]:
# 加入AI科学技术交流群探讨更多AI技术
# 第一次部署完执行，或者你要添加新的插件，编辑后执行

import os

%cd /home/jovyan/workspace
%env TF_CPP_MIN_LOG_LEVEL=1

def clone_project(git_url, des_name):
    if os.path.exists('/home/jovyan/workspace/stable-diffusion-webui/extensions/' + des_name):
        print(f"目录{des_name} 存在，中断clone")
        return
    !git clone {git_url} /home/jovyan/workspace/stable-diffusion-webui/extensions/{des_name}
        

!git clone -b v2.5 https://github.com/camenduru/stable-diffusion-webui
clone_project("https://github.com/etherealxx/batchlinks-webui", "batchlinks-webui")
clone_project("https://github.com/HeathWang/sd_extension_clean_images", "sd_extension_clean_images")
clone_project("https://github.com/HeathWang/sd-webui-infinite-image-browsing", "sd-webui-infinite-image-browsing")
clone_project("https://github.com/DominikDoom/a1111-sd-webui-tagcomplete", "a1111-sd-webui-tagcomplete")
clone_project("https://github.com/KohakuBlueleaf/a1111-sd-webui-lycoris", "a1111-sd-webui-lycoris")
clone_project("https://github.com/camenduru/sd-civitai-browser", "sd-civitai-browser")
clone_project("https://github.com/Bing-su/adetailer", "adetailer")
clone_project("https://github.com/hako-mikan/sd-webui-lora-block-weight", "sd-webui-lora-block-weight")
clone_project("'https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN'", "stable-diffusion-webui-localization-zh_CN")

%cd /home/jovyan/workspace/stable-diffusion-webui


!sed -i -e '''/from modules import launch_utils/a\import os''' /home/jovyan/workspace/stable-diffusion-webui/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /home/jovyan/workspace/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /home/jovyan/workspace/stable-diffusion-webui/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /home/jovyan/workspace/stable-diffusion-webui/modules/shared.py



/home/jovyan/workspace
env: TF_CPP_MIN_LOG_LEVEL=1
Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 24050, done.
remote: Total 24050 (delta 0), reused 0 (delta 0), pack-reused 24050
Receiving objects: 100% (24050/24050), 30.97 MiB | 45.64 MiB/s, done.
Resolving deltas: 100% (16844/16844), done.
Cloning into '/home/jovyan/workspace/stable-diffusion-webui/extensions/batchlinks-webui'...
remote: Enumerating objects: 540, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 540 (delta 110), reused 101 (delta 83), pack-reused 376
Receiving objects: 100% (540/540), 4.48 MiB | 34.01 MiB/s, done.
Resolving deltas: 100% (301/301), done.
Cloning into '/home/jovyan/workspace/stable-diffusion-webui/extensions/sd_extension_clean_images'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 55 (delta 24), reused 21 (delt

In [11]:
# 下载模型，VAE，lora等

import os
import subprocess
import re
import time
from concurrent.futures import ThreadPoolExecutor
import shutil

# 在这里填入大模型
def get_model_urls():
  selected_model_list = [
      # xxmix9realistic_v40.safetensors
      # 'https://civitai.com/api/download/models/102222',
      # ghostmix_v20Bakedvae
      'https://civitai.com/api/download/models/76907',
  ]
  
  return selected_model_list

model_urls = get_model_urls()

lora_urls = [
    'https://civitai.com/api/download/models/92986', # St. Louis
    'https://civitai.com/api/download/models/7932', # Girls' Frontline-OTs-14"lightning"
    'https://civitai.com/api/download/models/124906', # [LoRA] Cumbersome / 繁雑度 Concept (With dropout & noise version)
    'https://civitai.com/api/download/models/138117', # anxiang | 暗香 (fp16/lite)
    'https://civitai.com/api/download/models/140369', # Twinkling Twilight Taproom
]

lyco_model_urls=[
#FilmGirl 胶片风
'https://civitai.com/api/download/models/75069',
]


embeddings_urls = []

vae_model_urls=[
'https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.safetensors',
'https://huggingface.co/dector/vae-840000/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt',
'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt',
# 'https://huggingface.co/datasets/sukaka/sd_models_fp16/resolve/main/clearvae.vae.pt',
# 'https://huggingface.co/datasets/sukaka/sd_models_fp16/resolve/main/klF8Anime2.vae.pt',
# 'https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt',
]

ESRGAN_urls = [
# 'https://huggingface.co/FacehugmanIII/4x_foolhardy_Remacri/resolve/main/4x_foolhardy_Remacri.pth',
'https://huggingface.co/konohashinobi4/4xAnimesharp/resolve/main/4x-AnimeSharp.pth',
'https://huggingface.co/lokCX/4x-Ultrasharp/resolve/main/4x-UltraSharp.pth',
]

# 该变量为True，每次会删除大模型重新下载新的
should_re_download_big_model = False

def download_model():
    start_t = time.time()
    os.chdir('/home/jovyan/workspace')
    
    if os.path.exists('/home/jovyan/workspace/sd_DL/Stable-diffusion') and should_re_download_big_model:
        print('delete ckpt folder...')
        shutil.rmtree('/home/jovyan/workspace/sd_DL/Stable-diffusion')
    
    os.makedirs('sd_DL', exist_ok=True)
    os.chdir('sd_DL')
    os.makedirs('VAE', exist_ok=True)
    os.makedirs('Stable-diffusion', exist_ok=True)
    os.makedirs('Lora', exist_ok=True)
    os.makedirs('cn-model', exist_ok=True)
    os.makedirs('hypernetworks', exist_ok=True)
    os.makedirs('ESRGAN', exist_ok=True)
    os.makedirs('embeddings', exist_ok=True)
    os.makedirs('lyco', exist_ok=True)
    tasks = []
    tasks.extend(download_links(model_urls, 'Stable-diffusion'))
    tasks.extend(download_links(lora_urls, 'Lora'))
    tasks.extend(download_links(embeddings_urls, 'embeddings'))
    tasks.extend(download_links(ESRGAN_urls, 'ESRGAN'))
    tasks.extend(download_links(vae_model_urls, 'VAE'))
    tasks.extend(download_links(lyco_model_urls, 'lyco'))
    download_links_all(tasks)
    end_t = time.time()
    print("网络下载耗时：", end_t - start_t)


def download_link(link, target_folder):
    if link.startswith('https://huggingface.co/'):
        filename = re.search(r'[^/]+$', link).group(0)
        return f'aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M -d "{target_folder}" -o "{filename}" "{link}"'
    else:
        return f'aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M --remote-time -d "{target_folder}" "{link}"'

def download_links(links, target_folder):
    tasks = []
    for link in links:
        tasks.append(download_link(link, target_folder))
    return tasks

def download_links_all(tasks):
    with ThreadPoolExecutor(max_workers=5) as executor:
        for task in tasks:
            executor.submit(os.system, task)

download_model()

def get_directory_size(directory):
  total_size = 0

  for dirpath, dirnames, filenames in os.walk(directory):
      for filename in filenames:
          file_path = os.path.join(dirpath, filename)
          total_size += os.path.getsize(file_path)

  total_size = total_size / 1048576
  if total_size > 1024:
    total_size = total_size / 1024
    return "下载数据大小：{:.2f} GB".format(total_size)
  else:
    return "下载数据大小：{:.2f} MB".format(total_size)

download_size = get_directory_size('/home/jovyan/workspace/sd_DL')
print(download_size)


网络下载耗时： 1.5266480445861816
下载数据大小：4.33 GB


In [12]:
import concurrent.futures
import time
import subprocess
import os
import shutil

workspace = "/home/jovyan/workspace/"
my_work = "stable-diffusion-webui"

def create_symlinks(folder_paths, target_dir):
    # Create target directory if it doesn't exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    # Remove broken symlinks in target directory
    for filename in os.listdir(target_dir):
        target_path = os.path.join(target_dir, filename)
        if os.path.islink(target_path) and not os.path.exists(target_path):
            os.unlink(target_path)
    # Create new symlinks
    for source_path in folder_paths:
        if not os.path.exists(source_path):
            continue
        if os.path.isdir(source_path):
            for filename in os.listdir(source_path):
                source_file_path = os.path.join(source_path, filename)
                target_file_path = os.path.join(target_dir, filename)
                if not os.path.exists(target_file_path):
                    os.symlink(source_file_path, target_file_path)
                    print(f'Created symlink for {filename} in {target_dir}')
        else:
            filename = os.path.basename(source_path)
            target_file_path = os.path.join(target_dir, filename)
            if not os.path.exists(target_file_path):
                os.symlink(source_path, target_file_path)
                print(f'Created symlink for {filename} in {target_dir}')


def link_resource():
  start_t = time.time()

  create_symlinks(['/home/jovyan/workspace/sd_DL/Stable-diffusion'], f'/home/jovyan/workspace/{my_work}/models/Stable-diffusion')
  create_symlinks(['/home/jovyan/workspace/sd_DL/Lora'], f'/home/jovyan/workspace/{my_work}/models/Lora')
  create_symlinks(['/home/jovyan/workspace/sd_DL/embeddings'], f'/home/jovyan/workspace/{my_work}/embeddings')
  create_symlinks(['/home/jovyan/workspace/sd_DL/ESRGAN'], f'/home/jovyan/workspace/{my_work}/models/ESRGAN')
  create_symlinks(['/home/jovyan/workspace/sd_DL/VAE'], f'/home/jovyan/workspace/{my_work}/models/VAE')
  create_symlinks(['/home/jovyan/workspace/sd_DL/lyco'], f'/home/jovyan/workspace/{my_work}/models/LyCORIS')
  end_t = time.time()
  print("文件link耗时:", end_t - start_t)

link_resource()

def auto_complete_config_copy():
  if os.path.exists(f"{workspace}sd-configs2") is not True:
    !git clone https://github.com/HeathWang/sd-configs2 {workspace}sd-configs2
  else:
    shutil.rmtree(f'{workspace}sd-configs2')
    !git clone https://github.com/HeathWang/sd-configs2 {workspace}sd-configs2
  !cp -rf {workspace}sd-configs2/dist/ui-config.json {workspace}{my_work}
  !cp -rf {workspace}sd-configs2/dist/config.json {workspace}{my_work}
  !cp -rf {workspace}sd-configs2/dist/extensions/a1111-sd-webui-tagcomplete/tags/* {workspace}{my_work}/extensions/a1111-sd-webui-tagcomplete/tags

auto_complete_config_copy()

def downloadEmbeddings():
    
    %cd /home/jovyan/workspace/stable-diffusion-webui

    if os.path.exists(f'{workspace}{my_work}/embeddings'):
      shutil.rmtree(f'{workspace}{my_work}/embeddings')
    !git clone https://huggingface.co/nolanaatama/embeddings
## 下载embeddings，不需要就注释掉
downloadEmbeddings()

Created symlink for ghostmix_v20Bakedvae.safetensors in /home/jovyan/workspace/stable-diffusion-webui/models/Stable-diffusion
Created symlink for anxiang_L.safetensors in /home/jovyan/workspace/stable-diffusion-webui/models/Lora
Created symlink for st louis v2 epoch 5.safetensors in /home/jovyan/workspace/stable-diffusion-webui/models/Lora
Created symlink for bar1.safetensors in /home/jovyan/workspace/stable-diffusion-webui/models/Lora
Created symlink for 5blora5D20cumbersome2020E7B981E9.MjaS.safetensors in /home/jovyan/workspace/stable-diffusion-webui/models/Lora
Created symlink for Girls' Frontline-OTs-14[lightning]-V3.safetensors in /home/jovyan/workspace/stable-diffusion-webui/models/Lora
Created symlink for 4x-UltraSharp.pth in /home/jovyan/workspace/stable-diffusion-webui/models/ESRGAN
Created symlink for 4x-AnimeSharp.pth in /home/jovyan/workspace/stable-diffusion-webui/models/ESRGAN
Created symlink for orangemix.vae.pt in /home/jovyan/workspace/stable-diffusion-webui/models/VAE

In [13]:
import os
def get_directory_size(directory):
  total_size = 0

  for dirpath, dirnames, filenames in os.walk(directory):
      for filename in filenames:
          file_path = os.path.join(dirpath, filename)
          total_size += os.path.getsize(file_path)

  total_size = total_size / 1048576
  if total_size > 1024:
    total_size = total_size / 1024
    return f"{directory}" + "数据大小：{:.2f} GB".format(total_size)
  else:
    return f"{directory}" + "数据大小：{:.2f} MB".format(total_size)

download_size = get_directory_size('/home/jovyan/workspace')
print(download_size)

download_size = get_directory_size('/home/jovyan/workspace/sd_DL')
print(download_size)

download_size = get_directory_size('/home/jovyan/workspace/stable-diffusion-webui')
print(download_size)

/home/jovyan/workspace数据大小：8.75 GB
/home/jovyan/workspace/sd_DL数据大小：4.33 GB
/home/jovyan/workspace/stable-diffusion-webui数据大小：4.41 GB


In [ ]:
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue

%env TF_CPP_MIN_LOG_LEVEL=1


%cd /home/jovyan/workspace/stable-diffusion-webui
!python launch.py --listen --lowram --xformers --enable-insecure-extension-access --disable-console-progressbars --theme dark --gradio-queue --disable-safe-unpickle --disable-nan-check --no-hashing --port 8000


env: TF_CPP_MIN_LOG_LEVEL=1
/home/jovyan/workspace/stable-diffusion-webui
fatal: No names found, cannot describe anything.
Python 3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:39:03) 
[GCC 11.3.0]
Version: 1.5.1
Commit hash: 68f336bd994bed5442ad95bad6b6ad5564a5409a
Cloning Stable Diffusion into /home/jovyan/workspace/stable-diffusion-webui/repositories/stable-diffusion-stability-ai...
Cloning into '/home/jovyan/workspace/stable-diffusion-webui/repositories/stable-diffusion-stability-ai'...
remote: Enumerating objects: 574, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 574 (delta 244), reused 216 (delta 216), pack-reused 270
Receiving objects: 100% (574/574), 73.43 MiB | 49.37 MiB/s, done.
Resolving deltas: 100% (276/276), done.
Cloning Stable Diffusion XL into /home/jovyan/workspace/stable-diffusion-webui/repositories/generative-models...
Cloning into '/home/jovyan/workspace/stable-diffusion-webui/reposi

In [ ]:
# import shutil
# import os

# if os.path.exists('/home/jovyan/workspace/ui'):
#     shutil.rmtree('/home/jovyan/workspace/ui')
    
# if os.path.exists('/home/jovyan/workspace/stable-diffusion-webui'):
#     shutil.rmtree('/home/jovyan/workspace/stable-diffusion-webui')